# Part 1

In [1]:
from dataclasses import dataclass
import re

@dataclass
class Group:
    id_: int
    num_units: int
    hit_points: int
    attack_damage: int
    attack_type: str
    initiative: int
    weaknesses: set
    immunities: set
    army: str
    
    @property
    def effective_power(self):
        return self.num_units * self.attack_damage
    
    def __hash__(self):
        ''' Make sure that groups can be put into dicts and sets. '''
        return hash((self.id_, self.army))

group_re = re.compile(r'^(\d+) units each with (\d+) hit points (\(((weak|immune) '
                      r'to ([a-z, ]+))?;? ?((weak|immune) to ([a-z, ]+))?\))? ?'
                      r'with an attack that does (\d+) (\w+) damage at initiative (\d+)$')

def parse_group(line, id_, army):
    ''' Parse line of text and return a Group. '''
    match = group_re.match(line)
    if not match:
        raise Exception('Cannot parse group: {}'.format(line))
    num_units = int(match.group(1))
    hit_points = int(match.group(2))
    weaknesses, immunities = set(), set()
    if match.group(5) == 'weak':
        weaknesses = {w.strip() for w in match.group(6).split(',')} if match.group(4) else set()
    elif match.group(5) == 'immune':
        immunities = {i.strip() for i in match.group(6).split(',')} if match.group(4) else set()
    if match.group(8) == 'weak':
        weaknesses = {w.strip() for w in match.group(9).split(',')} if match.group(6) else set()
    elif match.group(8) == 'immune':
        immunities = {i.strip() for i in match.group(9).split(',')} if match.group(6) else set()
    attack_damage = int(match.group(10))
    attack_type = match.group(11)
    initiative = int(match.group(12))
    return Group(id_, num_units, hit_points, attack_damage, attack_type, initiative,
                 weaknesses, immunities, army)

In [2]:
g = parse_group('18 units each with 729 hit points (weak to fire; '
            'immune to cold, slashing) with an attack that does '
            '8 radiation damage at initiative 10', 1, 'Infection')
g

Group(id_=1, num_units=18, hit_points=729, attack_damage=8, attack_type='radiation', initiative=10, weaknesses={'fire'}, immunities={'cold', 'slashing'}, army='Infection')

In [3]:
g.effective_power

144

In [4]:
print(parse_group('17 units each with 5390 hit points (weak to radiation, bludgeoning) '
                  'with an attack that does 4507 fire damage at initiative 2', 1, 'Infection'))
print(parse_group('989 units each with 1274 hit points (immune to fire) '
                  'with an attack that does 25 slashing damage at initiative 3', 2, 'Infection'))
print(parse_group('989 units each with 1274 hit points (immune to fire; '
                  'weak to bludgeoning, slashing) with an attack that does '
                  '25 slashing damage at initiative 3', 3, 'Infection'))
print(parse_group('550 units each with 5068 hit points with an attack that '
                  'does 87 radiation damage at initiative 19', 4, 'Infection'))

Group(id_=1, num_units=17, hit_points=5390, attack_damage=4507, attack_type='fire', initiative=2, weaknesses={'radiation', 'bludgeoning'}, immunities=set(), army='Infection')
Group(id_=2, num_units=989, hit_points=1274, attack_damage=25, attack_type='slashing', initiative=3, weaknesses=set(), immunities={'fire'}, army='Infection')
Group(id_=3, num_units=989, hit_points=1274, attack_damage=25, attack_type='slashing', initiative=3, weaknesses={'slashing', 'bludgeoning'}, immunities={'fire'}, army='Infection')
Group(id_=4, num_units=550, hit_points=5068, attack_damage=87, attack_type='radiation', initiative=19, weaknesses=set(), immunities=set(), army='Infection')


In [5]:
import itertools

def parse_text(text):
    ''' Parse text and return two armies. '''
    immune_system = list()
    infection = list()
    current = None
    for line in text.strip().split('\n'):
        if line == 'Immune System:':
            current = immune_system
            id_counter = itertools.count()
            next(id_counter)
            army = 'Immune System'
        elif line == 'Infection:':
            current = infection
            id_counter = itertools.count()
            next(id_counter)
            army = 'Infection'
        elif line.strip() == '':
            pass
        else:
            current.append(parse_group(line.strip(), next(id_counter), army))
    return immune_system, infection

In [6]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)

In [7]:
print('immune system', immune_system)
print('infection', infection)

immune system [Group(id_=1, num_units=17, hit_points=5390, attack_damage=4507, attack_type='fire', initiative=2, weaknesses={'radiation', 'bludgeoning'}, immunities=set(), army='Immune System'), Group(id_=2, num_units=989, hit_points=1274, attack_damage=25, attack_type='slashing', initiative=3, weaknesses={'slashing', 'bludgeoning'}, immunities={'fire'}, army='Immune System')]
infection [Group(id_=1, num_units=801, hit_points=4706, attack_damage=116, attack_type='bludgeoning', initiative=1, weaknesses={'radiation'}, immunities=set(), army='Infection'), Group(id_=2, num_units=4485, hit_points=2961, attack_damage=12, attack_type='slashing', initiative=4, weaknesses={'fire', 'cold'}, immunities={'radiation'}, army='Infection')]


In [8]:
# Can we parse the entire problem input?
with open('input.txt') as input_:
    text = input_.read()
immune_system, infection = parse_text(text)
print(len(immune_system), len(infection))

10 10


In [46]:
from operator import itemgetter

def select_target(group, targets, debug=False):
    ''' Select a target for group or None if no target is available. '''
    target_order = [(get_damage(group, target), target.effective_power, target.initiative, target) 
                    for target in targets]
    target_order.sort(reverse=True)
    if debug:
        for damage, _, _, target in target_order:
            print('{} group {} would deal defending group {} {} damage'.format(
                group.army, group.id_, target.id_, damage))
    try:
        damage, _, _, target = target_order[0]
    except IndexError:
        # No targets left
        return None
    if damage > 0:
        targets.remove(target)
        return target
    else:
        return None

def get_damage(group, target):
    ''' Return damage amount inflicted when group attacks target. '''
    if group.attack_type in target.immunities:
        return 0
    elif group.attack_type in target.weaknesses:
        return group.effective_power * 2
    else:
        return group.effective_power
    
class WarIsDeadlocked(Exception):
    pass

def fight_round(immune_system, infection, debug=False):
    ''' Conduct one round of the fight using a list of immune system groups and
    a list of infection groups. Note: this modifies the input lists! '''
    if debug:
        print('Immune system:')
        for group in immune_system:
            print('Group {} contains {} units'.format(group.id_, group.num_units))
        print('Infection:')
        for group in infection:
            print('Group {} contains {} units'.format(group.id_, group.num_units))
        print()

    # Target selection phase
    groups = immune_system + infection
    target_order = [(g.effective_power, g.initiative, g) for g in groups]
    target_order.sort(key=itemgetter(0), reverse=True)
    targets = dict()
    immune_system_remaining = list(immune_system)
    infection_remaining = list(infection)

    for _, _, group in target_order:
        if group in immune_system:
            target = select_target(group, infection_remaining, debug)
        elif group in infection:
            target = select_target(group, immune_system_remaining, debug)
        else:
            raise Exception('Should not happen')
        targets[group] = target
        
    if debug:
        print()

    # After working on part 2, I realized that there are scenarios where the battle
    # never finishes because the armies get deadlocked: the only units that remain
    # are all immune to the attack types of the enemies. If we detect that condition,
    # then we should bail early.
    target_count = sum(0 if target is None else 1 for target in targets.values())
    if target_count == 0:
        raise WarIsDeadlocked('No targets can be found')

    # Attacking phase
    attack_order = [(g.initiative, g) for g in groups]
    attack_order.sort(key=itemgetter(0), reverse=True)
    total_units_destroyed = 0
    
    for _, group in attack_order:
        if group.num_units == 0:
            continue
        target = targets[group]
        if target is None:
            continue
        damage = get_damage(group, target)
        # Figure out how many units to remove.
        units_destroyed = min(damage // target.hit_points, target.num_units)
        total_units_destroyed += units_destroyed
        target.num_units -= units_destroyed
        if debug:
            group_name = 'Immune System' if group in immune_system else 'Infection'
            print('{} group {} attacks defending group {}, killing {} units'.format(
                group_name, group.id_, target.id_, units_destroyed))
    
    # Another way the war can get deadlocked: if none of the remaining units can do
    # enough damage to destroy any of the enemy units. We detect that condition and
    # raise an exception.
    if total_units_destroyed == 0:
        raise WarIsDeadlocked('No units can be destroyed')
    
    # Remove defeated units:
    for idx in range(len(immune_system)-1, -1, -1):
        if immune_system[idx].num_units == 0:
            del immune_system[idx]
    for idx in range(len(infection)-1, -1, -1):
        if infection[idx].num_units == 0:
            del infection[idx]

In [47]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)
# Note that the target selection round order of statements differs from the
# sample problem, but the values are correct.
fight_round(immune_system, infection, debug=True)

Immune system:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

Infection group 1 would deal defending group 1 185832 damage
Infection group 1 would deal defending group 2 185832 damage
Immune System group 1 would deal defending group 2 153238 damage
Immune System group 1 would deal defending group 1 76619 damage
Infection group 2 would deal defending group 2 107640 damage
Immune System group 2 would deal defending group 1 24725 damage

Infection group 2 attacks defending group 2, killing 84 units
Immune System group 2 attacks defending group 1, killing 4 units
Immune System group 1 attacks defending group 2, killing 51 units
Infection group 1 attacks defending group 1, killing 17 units


In [48]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 905 units
Infection:
Group 1 contains 797 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 184904 damage
Immune System group 2 would deal defending group 1 22625 damage
Immune System group 2 would deal defending group 2 22625 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 144 units


In [49]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 761 units
Infection:
Group 1 contains 793 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183976 damage
Immune System group 2 would deal defending group 1 19025 damage
Immune System group 2 would deal defending group 2 19025 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 143 units


In [50]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 618 units
Infection:
Group 1 contains 789 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183048 damage
Immune System group 2 would deal defending group 1 15450 damage
Immune System group 2 would deal defending group 2 15450 damage

Immune System group 2 attacks defending group 1, killing 3 units
Infection group 1 attacks defending group 2, killing 143 units


In [51]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 475 units
Infection:
Group 1 contains 786 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 182352 damage
Immune System group 2 would deal defending group 1 11875 damage
Immune System group 2 would deal defending group 2 11875 damage

Immune System group 2 attacks defending group 1, killing 2 units
Infection group 1 attacks defending group 2, killing 142 units


In [52]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 333 units
Infection:
Group 1 contains 784 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181888 damage
Immune System group 2 would deal defending group 1 8325 damage
Immune System group 2 would deal defending group 2 8325 damage

Immune System group 2 attacks defending group 1, killing 1 units
Infection group 1 attacks defending group 2, killing 142 units


In [53]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 191 units
Infection:
Group 1 contains 783 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181656 damage
Immune System group 2 would deal defending group 1 4775 damage
Immune System group 2 would deal defending group 2 4775 damage

Immune System group 2 attacks defending group 1, killing 1 units
Infection group 1 attacks defending group 2, killing 142 units


In [54]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 49 units
Infection:
Group 1 contains 782 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181424 damage
Immune System group 2 would deal defending group 1 1225 damage
Immune System group 2 would deal defending group 2 1225 damage

Immune System group 2 attacks defending group 1, killing 0 units
Infection group 1 attacks defending group 2, killing 49 units


In [55]:
# At this point, all immune system groups are killed.
print(immune_system)
# Only infection groups remain:
print(infection)

[]
[Group(id_=1, num_units=782, hit_points=4706, attack_damage=116, attack_type='bludgeoning', initiative=1, weaknesses={'radiation'}, immunities=set(), army='Infection'), Group(id_=2, num_units=4434, hit_points=2961, attack_damage=12, attack_type='slashing', initiative=4, weaknesses={'fire', 'cold'}, immunities={'radiation'}, army='Infection')]


In [56]:
sum(g.num_units for g in infection)

5216

In [57]:
import math

def fight(immune_system, infection, max_rounds=math.inf, debug=False):
    ''' A wrapper function that runs right_round() until one army is defeated. '''
    for n in itertools.count():
        fight_round(immune_system, infection, debug)
        if len(immune_system) == 0:
            print('Infection army won in {} rounds'.format(n))
            winning_army = infection
            break
        elif len(infection) == 0:
            print('Immune system won in {} rounds'.format(n))
            winning_army = immune_system
            break
        elif n >= max_rounds:
            print('Breaking after {} rounds'.format(max_rounds))
            return None
    return sum(g.num_units for g in winning_army)

In [58]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)
fight(immune_system, infection)

Infection army won in 7 rounds


5216

In [59]:
# The code above works for sample problem. Now try with input data.
with open('input.txt') as input_:
    text = input_.read()
immune_system, infection = parse_text(text)

In [60]:
%time fight(immune_system, infection)

Infection army won in 1370 rounds
CPU times: user 175 ms, sys: 2 ms, total: 177 ms
Wall time: 176 ms


16090

# Part 2

In [61]:
def boost(immune_system, amount):
    ''' Add a boost to all the units in the immune system army. '''
    for group in immune_system:
        group.attack_damage += amount

In [62]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)
boost(immune_system, 1570)
print(immune_system)

[Group(id_=1, num_units=17, hit_points=5390, attack_damage=6077, attack_type='fire', initiative=2, weaknesses={'radiation', 'bludgeoning'}, immunities=set(), army='Immune System'), Group(id_=2, num_units=989, hit_points=1274, attack_damage=1595, attack_type='slashing', initiative=3, weaknesses={'slashing', 'bludgeoning'}, immunities={'fire'}, army='Immune System')]


In [63]:
fight(immune_system, infection)

Immune system won in 40 rounds


51

In [64]:
def check_boost(amount, max_rounds=math.inf):
    ''' Run the fight with the given boost. '''
    with open('input.txt') as input_:
        text = input_.read()
    immune_system, infection = parse_text(text)
    boost(immune_system, amount)
    return fight(immune_system, infection, max_rounds)

In [65]:
check_boost(0)

Infection army won in 1370 rounds


16090

In [66]:
check_boost(1000)

Immune system won in 31 rounds


29516

Okay, so 1000 is big enough to get the immune system to win. Now I will do a manual binary search to find the lowest number. (Searching manually is probably faster than trying to code and test a perfect binary search algorithm.)

In [67]:
check_boost(500)

Immune system won in 61 rounds


28079

In [68]:
check_boost(250)

Immune system won in 116 rounds


25820

In [69]:
check_boost(125)

Immune system won in 217 rounds


21922

In [70]:
check_boost(62)

Immune system won in 445 rounds


14091

In [71]:
%time check_boost(31)

Immune system won in 924 rounds
CPU times: user 119 ms, sys: 1.93 ms, total: 121 ms
Wall time: 120 ms


9526

In [72]:
# At this point, my binary search plan hits a snag: I find out that when the
# units are evenly matched, the war can result in a deadlock: neither army can
# inflict any damage because the remain units are all immune to the attackers.
# I added code that checks for deadlock and raises an exception. Notice the 
# difference between these two boost values: the first one succeeds and the
# second reaches deadlock. Without deadlock detection, the second one would
# infinite loop.
%time check_boost(27)
%time check_boost(26)

Immune system won in 1163 rounds
CPU times: user 138 ms, sys: 2.19 ms, total: 140 ms
Wall time: 138 ms


WarIsDeadlocked: No targets can be found

In [78]:
%%time
# So I know the optimal value is 27 or less. I'll run all of these values
# to see if the immune system wins in any other scenario.
for amount in range(0,28):
    print('Amount', amount)
    try:
        outcome = check_boost(amount, max_rounds=1_000_000)
    except WarIsDeadlocked as exc:
        print('War is deadlocked: {}'.format(exc))
        continue
    print('Outcome', outcome)

Amount 0
Infection army won in 1370 rounds
Outcome 16090
Amount 1
Infection army won in 1369 rounds
Outcome 15740
Amount 2
Infection army won in 1598 rounds
Outcome 14993
Amount 3
Infection army won in 1602 rounds
Outcome 14631
Amount 4
Infection army won in 1602 rounds
Outcome 14341
Amount 5
Infection army won in 1604 rounds
Outcome 13857
Amount 6
Infection army won in 1612 rounds
Outcome 13346
Amount 7
Infection army won in 1613 rounds
Outcome 12933
Amount 8
Infection army won in 1608 rounds
Outcome 12524
Amount 9
Infection army won in 1609 rounds
Outcome 12121
Amount 10
Infection army won in 1612 rounds
Outcome 11945
Amount 11
Infection army won in 1768 rounds
Outcome 11476
Amount 12
Infection army won in 1819 rounds
Outcome 10838
Amount 13
Infection army won in 2232 rounds
Outcome 9893
Amount 14
Infection army won in 2901 rounds
Outcome 8414
Amount 15
War is deadlocked: No targets can be found
Amount 16
War is deadlocked: No targets can be found
Amount 17
War is deadlocked: No targ

In [82]:
# The immune system doesn't win until boost is 27, so I submitted 8304 as
# an answer but that answer is too high. Could it be that a deadlock counts
# as a win? The first deadlock occurs at boost==15.
with open('input.txt') as input_:
    text = input_.read()
immune_system, infection = parse_text(text)
boost(immune_system, 15)
fight(immune_system, infection)

WarIsDeadlocked: No targets can be found

In [84]:
# Nope, 2722 is too low.
print(immune_system)
print(infection)

[Group(id_=5, num_units=2722, hit_points=6619, attack_damage=21, attack_type='bludgeoning', initiative=12, weaknesses=set(), immunities={'fire', 'slashing'}, army='Immune System')]
[Group(id_=2, num_units=4684, hit_points=16609, attack_damage=6, attack_type='fire', initiative=2, weaknesses=set(), immunities={'radiation', 'slashing', 'bludgeoning'}, army='Infection'), Group(id_=4, num_units=1046, hit_points=30069, attack_damage=29, attack_type='fire', initiative=7, weaknesses=set(), immunities={'cold', 'bludgeoning'}, army='Infection')]
